In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import re
import math

In [ ]:
df = pd.read_csv("./train.csv")

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
df['Fare'].fillna(df['Fare'].median(), inplace = True)
df['Age'].fillna(df['Age'].median(),inplace=True)

In [ ]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [ ]:
df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)
df['Age'] = df['Age'].apply(lambda x: 0.0 if x <= 16 else (1.0 if x <= 26 else (2.0 if x <= 36 else 3.0)))
df['Fare'] = df['Fare'].apply(lambda x: 0.0 if x <= 17 else (1.0 if x <= 30 else (2.0 if x <= 100 else 3.0)))
df['Embarked'] = df['Embarked'].apply(lambda x: 0 if x == 'S' else (1 if x == 'C' else 2))
df['FamilySize'] = (df['SibSp'] + df['Parch']) / 2.5
df['AgeTimesPclass'] = df['Age'] * df['Pclass']
df['Mr'] = df['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Mr.' else 0)
df['Mrs'] = df['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Mrs.' else 0)
df['Ms'] = df['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Ms.' else 0)
df['Dr'] = df['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Dr.' else 0)
df['Alone'] = df['FamilySize'].apply(lambda x:1 if x==0 else 0)
drop_features = ['Name','SibSp','Parch','Ticket','Cabin']
df.drop(drop_features,inplace=True,axis=1)

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,AgeTimesPclass,Mr,Mrs,Ms,Dr,Alone
0,1,0,3,0,1.0,0.0,0,0.4,3.0,1,0,0,0,0
1,2,1,1,1,3.0,2.0,1,0.4,3.0,0,1,0,0,0
2,3,1,3,1,1.0,0.0,0,0.0,3.0,0,0,0,0,1
3,4,1,1,1,2.0,2.0,0,0.4,2.0,0,1,0,0,0
4,5,0,3,0,2.0,0.0,0,0.0,6.0,1,0,0,0,1


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.loc[:,'Pclass':],df.Survived,\
                                                          test_size=0.1)

In [ ]:
rf_model = RandomForestClassifier(criterion='gini', n_estimators=1000,
                               max_depth=5, min_samples_split=4,
                               min_samples_leaf=5, max_features='auto',
                               oob_score=True, random_state=0)
rf_model.fit(x_train,y_train)
rf_model_predictions = rf_model.predict(x_test)
print(classification_report(y_test,rf_model_predictions))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90        57
           1       0.82      0.85      0.84        33

    accuracy                           0.88        90
   macro avg       0.87      0.87      0.87        90
weighted avg       0.88      0.88      0.88        90



In [ ]:
rf_model.score(x_test,y_test)

0.8777777777777778

In [ ]:
kf = KFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
X = df.loc[:,'Pclass':]
y = df.loc[:,'Survived']
scoring = 'accuracy'
results = cross_val_score(rf_model, X.values, y.values, cv=kf, n_jobs=1, scoring=scoring)
results

array([0.85555556, 0.80898876, 0.84269663, 0.80898876, 0.86516854,
       0.86516854, 0.78651685, 0.83146067, 0.79775281, 0.8988764 ])

In [ ]:
print(results.mean())

0.8361173533083643


In [ ]:
df_test = pd.read_csv("./test.csv")
df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace = True)
df_test['Fare'].fillna(df_test['Fare'].median(), inplace = True)
df_test['Age'].fillna(df_test['Age'].median(),inplace=True)
df_test['Sex'] = df_test['Sex'].apply(lambda x: 0 if x == 'male' else 1)
df_test['Age'] = df_test['Age'].apply(lambda x: 0.0 if x <= 16 else (1.0 if x <= 26 else (2.0 if x <= 36 else 3.0)))
df_test['Fare'] = df_test['Fare'].apply(lambda x: 0.0 if x <= 17 else (1.0 if x <= 30 else (2.0 if x <= 100 else 3.0)))
df_test['Embarked'] = df_test['Embarked'].apply(lambda x: 0 if x == 'S' else (1 if x == 'C' else 2))
df_test['FamilySize'] = (df_test['SibSp'] + df_test['Parch']) / 2.5
df_test['AgeTimesPclass'] = df_test['Age'] * df_test['Pclass']
df_test['Mr'] = df_test['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Mr.' else 0)
df_test['Mrs'] = df_test['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Mrs.' else 0)
df_test['Ms'] = df_test['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Ms.' else 0)
df_test['Dr'] = df_test['Name'].apply(lambda x: 1 if re.findall('[A-Za-z]+\.',x)[0] == 'Dr.' else 0)
df_test['Alone'] = df_test['FamilySize'].apply(lambda x:1 if x==0 else 0)
drop_features = ['Name','SibSp','Parch','Ticket','Cabin']
df_test.drop(drop_features,inplace=True,axis=1)

predictions_for_submission = rf_model.predict(df_test.loc[:,"Pclass":])
df_submission = df_test[['PassengerId']].copy()
df_submission['Survived'] = predictions_for_submission
df_submission.to_csv("submission_v1.csv")